In [ ]:
! pip install -r requirements.txt

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

#model_id = "google/gemma-2b-it"

model_id = "facebook/bart-large-mnli"
model_id = "distilbert/distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id)

model.eval()

device = 'cpu' 

In [ ]:
chat = [
        {"role": "user", "content": prompt}
    ]
    formatted_prompt = tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        add_generation_prompt=True,
    )

In [31]:
def inference(question: str, context: str):

    if context == None or context == "":
        prompt = f"""Give a detailed answer to the following question. Question: {question}"""
    else:
        prompt = f"""Using the information contained in the context, give a detailed answer to the question.
            Context: {context}.
            Question: {question}"""
    
    system_prompt = "You are an AI assistant."
    formatted_prompt = f"[INST] <<SYS>>\n{system_prompt}\n<</SYS>>[INST] {question} [/INST]"
    
    tokenizer.padding_side = "left"
    tokenizer.pad_token_id = tokenizer.eos_token_id
    
    inputs = tokenizer.encode(
        formatted_prompt, add_special_tokens=False, return_tensors="pt"
    )
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_length=100
            #max_new_tokens=250,
            #do_sample=False,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #response = response[len(formatted_prompt) :]  # remove input prompt from reponse
    response = response.replace("<eos>", "")  # remove eos token
    return response


question = "Where is Paris?"
print(inference(question=question, context=""))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[INST] <<SYS>>
You are an AI assistant.
<</SYS>>[INST] Where is Paris? [/INST]
<</SYS>>[INST] Where is Paris? [/INST]
<</SYS>>[INST] Where is Paris? [/INST]
<</SYS>>[INST] Where is Paris? [/INST]
<</SYS>>[INST] Where is Paris? [/INST]
<</SYS>>


In [5]:
import ollama

ollama.generate(model='gemma:2b', prompt=question)

{'model': 'gemma:2b',
 'created_at': '2024-05-22T02:41:02.0002534Z',
 'response': 'A transformer is an electrical device that transfers energy from one circuit to another through induction. It is a passive device, meaning that it does not generate or consume electrical energy itself.\n\n**Key characteristics of a transformer:**\n\n* **Inductance:** The ability of a transformer to store and release energy in an alternating current circuit.\n* **Voltage:** The amount of voltage supplied to the primary circuit.\n* **Current:** The amount of current flowing through the primary and secondary circuits.\n* **Power:** The rate at which energy is transferred from the primary to the secondary circuit.\n* **Turns ratio:** The ratio of the number of turns in the primary to the number of turns in the secondary circuit.\n\n**How a transformer works:**\n\nA transformer works based on the principle of electromagnetic induction. When an alternating current is applied to the primary circuit, it creates 

In [7]:
from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("files/LLMclinical.pdf")
]
pages = []
for loader in loaders:
    pages.extend(loader.load())

In [8]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=12)
docs = text_splitter.split_documents(pages)

In [10]:
print(docs[2].page_content)

elle Barral1, 
Christopher Semturs1, Alan Karthikesalingam1,5 ✉ & Vivek Natarajan1,5 ✉
Large language models (LLMs) have demonstrated impressive capabilities, but the 
bar for clinical applications is high. Attempts to assess the clinical knowledge of 
models typically rely on automated evaluations based on limited benchmarks. Here, to address these limitations, we present MultiMedQA, a benchmark combining six 
existing medical question answering datasets spanning professional medicine, 
research and consumer queries and a new dataset of medical questions


In [11]:
import numpy as np
from langchain_community.embeddings import (
    HuggingFaceEmbeddings
)
encoder = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2', model_kwargs = {'device': "cpu"})

In [32]:
embeddings1 = encoder.embed_query("RAG")
embeddings2 = encoder.embed_query(docs[0].page_content)
print(np.dot(embeddings1, embeddings2))

0.02857219219547718


In [33]:
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

faiss_db = FAISS.from_documents(docs, encoder, distance_strategy=DistanceStrategy.DOT_PRODUCT)

In [40]:
question = "What is MultiMedQA?"
retrieved_docs = faiss_db.similarity_search(question, k=8)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)
print(context)

 and metadata: 
presence of labels or explanations and their sources. A summary of 
MultiMedQA is presented in Extended Data Table 1.
Although MedMCQA, PubMedQA, LiveQA, and MedicationQA 
provide reference long-form answers or explanations, we do not use them in this work. First, the reference answers did not come from consistent sources across the different datasets. Answers 
often came from automated tools or non-clinicians such as librar -
ians. The construction of the reference answers and explanations 
in these pioneering datasets was not

MultiMedQA benchmark. MultiMedQA includes medical exams 
and research datasets with multiple-choice answers and consumer 
medical question datasets with long-form answers. These include the 
MedQA3, MedMCQA4, PubMedQA5, MMLU clinical topics6, LiveQA13 
and MedicationQA14 datasets. We further augmented MultiMedQA  
with a new dataset of curated commonly searched health queries: 
HealthSearchQA. All the datasets are in the English language and we 

In [ ]:
torch.cuda.empty_cache()
print(inference(question=question, context=context))

In [36]:
def infer(question: str, context: str):

    if context == None or context == "":
        prompt = f"""Give a detailed answer to the following question. Question: {question}"""
    else:
        prompt = f"""Using the information contained in the context, give a detailed answer to the question.
            Context: {context}.
            Question: {question}"""    
    print(prompt)
    response = ollama.generate(model='gemma:2b', prompt=prompt)
   
    return response

In [41]:
infer(question=question, context=context)

{'model': 'gemma:2b',
 'created_at': '2024-05-22T04:51:26.2143812Z',
 'response': '**MultiMedQA is a benchmark comprising seven medical question-answering datasets, including six existing datasets: MedQA3, MedMCQA4, PubMedQA5, LiveQA13, MedicationQA** between Med-PaLM’s responses to MultiMedQA consumer questions and the PaLM training corpus and observed no overlap.\n\n**Sources of Labels and Explanations**\nThe context does not specify the sources of the labels and explanations for the datasets in the benchmark. Therefore, I cannot provide a detailed answer to this question from the context.',
 'done': True,
 'context': [106,
  1645,
  108,
  15325,
  573,
  2113,
  13614,
  575,
  573,
  4807,
  235269,
  2734,
  476,
  11352,
  3448,
  577,
  573,
  2872,
  235265,
  108,
  149,
  2930,
  235292,
  139,
  639,
  29182,
  235292,
  235248,
  108,
  75589,
  576,
  17737,
  689,
  44353,
  578,
  1024,
  8269,
  235265,
  586,
  13367,
  576,
  235248,
  108,
  9886,
  4567,
  39213,
 

In [ ]:
print("For this answer I used the following documents:")
for doc in retrieved_docs:
    print(doc.metadata)